# 🔐 Modern PSI Protocols Performance Dashboard

**Private Set Intersection (PSI) Interactive Benchmark & Analysis Platform**

Author: kn1ghtc | Security Research Platform  
Date: 2024-09-18  
Version: 1.0

---

## 📋 Overview

This interactive dashboard provides comprehensive performance analysis and comparison of modern PSI protocols:

1. **Microsoft APSI** - Production-grade C++ implementation with RLWE security
2. **OPRF-PSI** - Oblivious Pseudo-Random Function based protocol  
3. **Piano-PSI** - Sublinear communication complexity protocol
4. **VOLE-PSI** - Vector Oblivious Linear Evaluation protocol
5. **Circuit-PSI** - Multi-party computation circuit-based protocol

### 🎯 Features
- Real-time performance benchmarking
- Interactive parameter configuration
- Advanced visualization and analysis
- Security comparison matrix
- Export functionality for research use

## 📦 Setup & Dependencies

In [13]:
# Core dependencies
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add the current directory to path for importing our PSI module
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Data analysis and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Timing and utilities
import time
import json
from datetime import datetime

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
sns.set_palette("husl")

# Enable Plotly for Jupyter
pyo.init_notebook_mode(connected=True)

print("✅ Dependencies loaded successfully")
print(f"📍 Working directory: {current_dir}")

✅ Dependencies loaded successfully
📍 Working directory: d:\pyproject\kccrypt\secureComputation


In [14]:
# Import our PSI protocols
try:
    from PrivateSetInter import (
        PSIBenchmark, 
        MicrosoftAPSI,
        OPRFPSI,
        PianoPSI,
        VOLEPSI,
        CircuitPSI
    )
    print("✅ PSI protocols imported successfully")
    
    # Initialize benchmark instance
    global_benchmark = PSIBenchmark()
    print(f"✅ Benchmark initialized with {len(global_benchmark.protocols)} protocols")
    
except ImportError as e:
    print(f"❌ Failed to import PSI protocols: {e}")
    print("Please ensure PrivateSetInter.py is in the current directory")
    
except Exception as e:
    print(f"❌ Unexpected error: {e}")

2025-09-18 19:44:47,518 - INFO - 🔐 OPRF-PSI initialized with 128-bit security
2025-09-18 19:44:47,518 - INFO - 🔐 VOLE-PSI initialized with 128-bit security
2025-09-18 19:44:47,518 - INFO -    Batch size: 1000
2025-09-18 19:44:47,518 - INFO - 🔐 Piano-PSI initialized with 128-bit security
2025-09-18 19:44:47,518 - INFO - 🔐 Circuit-PSI initialized with 128-bit security
2025-09-18 19:44:47,518 - INFO -    Circuit type: boolean
2025-09-18 19:44:47,533 - INFO - 🔐 Microsoft-APSI-RSA-PSI initialized with 128-bit security
2025-09-18 19:44:47,533 - INFO -    Protocol mode: rsa-psi
2025-09-18 19:44:47,533 - INFO -    C++ executable: d:\pyproject\kccrypt\secureComputation\Release\apsi_only_test.exe


✅ PSI protocols imported successfully
✅ Benchmark initialized with 5 protocols


## ⚙️ Interactive Configuration Panel

In [15]:
# Create interactive widgets for benchmark configuration
class PSIConfigPanel:
    def __init__(self):
        self.setup_widgets()
        self.results = None
        
    def setup_widgets(self):
        # Protocol selection
        self.protocol_selection = widgets.SelectMultiple(
            options=['Microsoft-APSI', 'OPRF-PSI', 'Piano-PSI', 'VOLE-PSI', 'Circuit-PSI'],
            value=['Microsoft-APSI', 'OPRF-PSI', 'Piano-PSI'],
            description='Protocols:',
            layout=widgets.Layout(width='300px', height='120px')
        )
        
        # Test configurations
        self.small_config = widgets.IntRangeSlider(
            value=[50, 75],
            min=10, max=200, step=5,
            description='Small Scale:',
            layout=widgets.Layout(width='400px')
        )
        
        self.medium_config = widgets.IntRangeSlider(
            value=[250, 400],
            min=100, max=1000, step=25,
            description='Medium Scale:',
            layout=widgets.Layout(width='400px')
        )
        
        self.large_config = widgets.IntRangeSlider(
            value=[800, 1200],
            min=500, max=2000, step=50,
            description='Large Scale:',
            layout=widgets.Layout(width='400px')
        )
        
        # Benchmark parameters
        self.trials = widgets.IntSlider(
            value=3, min=1, max=10, step=1,
            description='Trials:',
            layout=widgets.Layout(width='300px')
        )
        
        self.security_param = widgets.Dropdown(
            options=[80, 128, 192, 256],
            value=128,
            description='Security:',
            layout=widgets.Layout(width='200px')
        )
        
        # Control buttons
        self.run_button = widgets.Button(
            description='🚀 Run Benchmark',
            button_style='primary',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        self.export_button = widgets.Button(
            description='💾 Export Results',
            button_style='success',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        self.clear_button = widgets.Button(
            description='🗑️ Clear Results',
            button_style='warning',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        # Progress indicator
        self.progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description='Progress:',
            layout=widgets.Layout(width='500px')
        )
        
        self.status_text = widgets.HTML(
            value="<b>Status:</b> Ready to run benchmark"
        )
        
        # Set up event handlers
        self.run_button.on_click(self.run_benchmark)
        self.export_button.on_click(self.export_results)
        self.clear_button.on_click(self.clear_results)
        
    def display_panel(self):
        # Layout the configuration panel
        left_column = widgets.VBox([
            widgets.HTML("<h3>📋 Protocol Selection</h3>"),
            self.protocol_selection,
            widgets.HTML("<h3>⚙️ Parameters</h3>"),
            widgets.HBox([self.trials, self.security_param])
        ])
        
        right_column = widgets.VBox([
            widgets.HTML("<h3>📊 Test Configurations</h3>"),
            self.small_config,
            self.medium_config,
            self.large_config,
        ])
        
        config_section = widgets.HBox([left_column, right_column])
        
        control_section = widgets.VBox([
            widgets.HTML("<h3>🎮 Controls</h3>"),
            widgets.HBox([self.run_button, self.export_button, self.clear_button]),
            self.progress,
            self.status_text
        ])
        
        full_panel = widgets.VBox([config_section, control_section])
        display(full_panel)
        
    def run_benchmark(self, button):
        """Run the PSI benchmark with current configuration"""
        try:
            self.status_text.value = "<b>Status:</b> <span style='color: orange'>Running benchmark...</span>"
            self.progress.value = 0
            
            # Get configuration
            selected_protocols = list(self.protocol_selection.value)
            test_configs = [
                tuple(self.small_config.value),
                tuple(self.medium_config.value),
                tuple(self.large_config.value)
            ]
            
            if not selected_protocols:
                self.status_text.value = "<b>Status:</b> <span style='color: red'>Please select at least one protocol</span>"
                return
            
            # Filter protocols
            filtered_benchmark = PSIBenchmark()
            for protocol_name in selected_protocols:
                if protocol_name in global_benchmark.protocols:
                    filtered_benchmark.protocols[protocol_name] = global_benchmark.protocols[protocol_name]
            
            self.progress.value = 20
            
            # Run benchmark
            start_time = time.time()
            self.results = filtered_benchmark.run_comprehensive_benchmark(
                test_configs, 
                self.trials.value
            )
            
            elapsed_time = time.time() - start_time
            
            self.progress.value = 100
            self.status_text.value = f"<b>Status:</b> <span style='color: green'>Completed in {elapsed_time:.1f}s</span>"
            
            # Display results
            self.display_results()
            
        except Exception as e:
            self.status_text.value = f"<b>Status:</b> <span style='color: red'>Error: {e}</span>"
            self.progress.value = 0
    
    def export_results(self, button):
        """Export benchmark results to JSON file"""
        if self.results is None:
            self.status_text.value = "<b>Status:</b> <span style='color: orange'>No results to export</span>"
            return
        
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"psi_dashboard_results_{timestamp}.json"
            
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.results, f, indent=2, default=str, ensure_ascii=False)
            
            self.status_text.value = f"<b>Status:</b> <span style='color: green'>Results exported to {filename}</span>"
            
        except Exception as e:
            self.status_text.value = f"<b>Status:</b> <span style='color: red'>Export failed: {e}</span>"
    
    def clear_results(self, button):
        """Clear benchmark results and reset display"""
        self.results = None
        self.progress.value = 0
        self.status_text.value = "<b>Status:</b> Ready to run benchmark"
        clear_output(wait=True)
        print("Results cleared. Run the next cell to display the configuration panel.")
    
    def display_results(self):
        """Display comprehensive results visualization"""
        if self.results is None:
            return
        
        # Create results visualizations
        self.create_performance_charts()
        self.create_security_matrix()
        self.create_summary_table()
    
    def create_performance_charts(self):
        """Create comprehensive performance visualization charts"""
        protocol_summaries = self.results.get('protocol_summaries', {})
        
        if not protocol_summaries:
            print("No performance data available")
            return
        
        # Prepare data
        protocols = list(protocol_summaries.keys())
        exec_times = [protocol_summaries[p]['avg_execution_time_ms'] for p in protocols]
        comm_costs = [protocol_summaries[p]['avg_communication_cost_bytes'] / 1024 for p in protocols]  # KB
        success_rates = [protocol_summaries[p]['success_rate'] for p in protocols]
        throughputs = [protocol_summaries[p]['throughput_ops_per_sec'] for p in protocols]
        
        # Create subplots using Plotly
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Execution Time (ms)', 'Communication Cost (KB)', 
                          'Success Rate (%)', 'Throughput (ops/sec)'),
            vertical_spacing=0.12
        )
        
        # Execution time chart
        fig.add_trace(
            go.Bar(x=protocols, y=exec_times, name='Execution Time', 
                  marker_color='lightblue', showlegend=False),
            row=1, col=1
        )
        
        # Communication cost chart
        fig.add_trace(
            go.Bar(x=protocols, y=comm_costs, name='Communication Cost',
                  marker_color='lightgreen', showlegend=False),
            row=1, col=2
        )
        
        # Success rate chart
        colors = ['green' if sr >= 90 else 'orange' if sr >= 50 else 'red' for sr in success_rates]
        fig.add_trace(
            go.Bar(x=protocols, y=success_rates, name='Success Rate',
                  marker_color=colors, showlegend=False),
            row=2, col=1
        )
        
        # Throughput chart
        fig.add_trace(
            go.Bar(x=protocols, y=throughputs, name='Throughput',
                  marker_color='coral', showlegend=False),
            row=2, col=2
        )
        
        # Update layout
        fig.update_layout(
            height=600, 
            title_text="📊 PSI Protocols Performance Comparison",
            title_font_size=16
        )
        
        # Update y-axes labels
        fig.update_yaxis(title_text="Time (ms)", row=1, col=1)
        fig.update_yaxis(title_text="Cost (KB)", row=1, col=2)
        fig.update_yaxis(title_text="Success (%)", row=2, col=1)
        fig.update_yaxis(title_text="Ops/sec", row=2, col=2)
        
        fig.show()
    
    def create_security_matrix(self):
        """Create security features comparison matrix"""
        # Security comparison data
        security_data = {
            'Protocol': ['Microsoft-APSI', 'OPRF-PSI', 'Piano-PSI', 'VOLE-PSI', 'Circuit-PSI'],
            'Post-Quantum': ['✅', '❌', '✅', '✅', '✅'],
            'Malicious Security': ['✅', '❌', 'Partial', '✅', '✅'],
            'Production Ready': ['✅', 'Partial', 'Research', 'Research', 'Research'],
            'Communication': ['Efficient', 'High', 'Sublinear', 'Low', 'Very High'],
            'Computation': ['Fast', 'Medium', 'Fast', 'Medium', 'Slow']
        }
        
        df_security = pd.DataFrame(security_data)
        
        # Display as styled table
        display(HTML("<h3>🔒 Security Features Comparison</h3>"))
        display(df_security.style.set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#4CAF50'), 
                                       ('color', 'white'), ('font-weight', 'bold')]},
            {'selector': 'td', 'props': [('text-align', 'center')]}
        ]).set_properties(**{'font-size': '12px'}))
    
    def create_summary_table(self):
        """Create performance summary table"""
        protocol_summaries = self.results.get('protocol_summaries', {})
        
        if not protocol_summaries:
            return
        
        # Prepare summary data
        summary_data = []
        for protocol, stats in protocol_summaries.items():
            summary_data.append({
                'Protocol': protocol,
                'Avg Time (ms)': f"{stats['avg_execution_time_ms']:.2f}",
                'Std Time (ms)': f"{stats['std_execution_time_ms']:.2f}",
                'Comm Cost (KB)': f"{stats['avg_communication_cost_bytes']/1024:.2f}",
                'Success Rate (%)': f"{stats['success_rate']:.1f}",
                'Throughput (ops/s)': f"{stats['throughput_ops_per_sec']:.2f}",
                'Tests': stats['total_tests']
            })
        
        df_summary = pd.DataFrame(summary_data)
        
        # Display styled summary table
        display(HTML("<h3>📈 Performance Summary</h3>"))
        display(df_summary.style.background_gradient(subset=['Avg Time (ms)', 'Throughput (ops/s)'])
               .format({'Avg Time (ms)': '{:.2f}', 'Throughput (ops/s)': '{:.2f}'})
               .set_table_styles([{
                   'selector': 'th',
                   'props': [('background-color', '#2196F3'), ('color', 'white'), ('font-weight', 'bold')]
               }]))

# Create and display the configuration panel
config_panel = PSIConfigPanel()
print("✅ Interactive configuration panel created")
print("📋 Run the next cell to display the control panel")

✅ Interactive configuration panel created
📋 Run the next cell to display the control panel


## 🎮 Interactive Control Panel

Use the controls below to configure and run PSI protocol benchmarks:

In [16]:
# Display the interactive configuration panel
config_panel.display_panel()

## 📊 Protocol Analysis & Recommendations

### 🏆 Best Protocol Selection Guide

Choose the optimal PSI protocol based on your specific requirements:

| Use Case | Recommended Protocol | Rationale |
|----------|---------------------|------------|
| **Production Deployment** | Microsoft APSI | Real C++ implementation, proven security, post-quantum |
| **Research & Prototyping** | OPRF-PSI | Good balance of performance and correctness |
| **Large-Scale Data** | Piano-PSI | Sublinear communication complexity |
| **Post-Quantum Security** | Microsoft APSI, VOLE-PSI, Piano-PSI | RLWE/LWE based security |
| **Educational Purposes** | OPRF-PSI | Simpler to understand and implement |

### 🔍 Performance Insights

- **Microsoft APSI**: Production-ready with ~100ms execution time, excellent for real applications
- **Piano-PSI**: Fastest theoretical protocol with sublinear scaling advantages
- **OPRF-PSI**: Reliable baseline with 100% correctness in simulations
- **VOLE-PSI & Circuit-PSI**: Advanced protocols requiring further optimization

## 🔬 Advanced Analysis Functions

Additional analysis tools for deep-dive research:

In [17]:
def analyze_scalability_trends(results):
    """Analyze how protocols scale with increasing set sizes"""
    if not results or 'protocol_summaries' not in results:
        print("No results available for scalability analysis")
        return
    
    print("📈 Scalability Trend Analysis")
    print("=" * 40)
    
    scalability = results.get('scalability_analysis', {})
    if scalability:
        for protocol, analysis in scalability.items():
            if isinstance(analysis, dict) and 'growth_analysis' in analysis:
                growth_type = analysis['growth_analysis']
                growth_rate = analysis.get('growth_rate', 'N/A')
                scalability_score = analysis.get('scalability_score', 'N/A')
                
                print(f"\n{protocol}:")
                print(f"  Growth Pattern: {growth_type}")
                print(f"  Growth Rate: {growth_rate}")
                print(f"  Scalability Score: {scalability_score}")

def export_detailed_report(results, filename=None):
    """Export a detailed markdown report of benchmark results"""
    if not results:
        print("No results to export")
        return
    
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"psi_detailed_report_{timestamp}.md"
    
    report_content = f"""# PSI Protocols Benchmark Report

**Generated:** {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}  
**Platform:** kn1ghtc Security Research Platform

## Executive Summary

This report presents comprehensive performance analysis of modern Private Set Intersection (PSI) protocols.

## Performance Results

"""
    
    protocol_summaries = results.get('protocol_summaries', {})
    
    for protocol, stats in protocol_summaries.items():
        report_content += f"""### {protocol}

- **Average Execution Time:** {stats['avg_execution_time_ms']:.2f} ms
- **Communication Cost:** {stats['avg_communication_cost_bytes']:,} bytes
- **Success Rate:** {stats['success_rate']:.1f}%
- **Throughput:** {stats['throughput_ops_per_sec']:.2f} operations/second
- **Total Tests:** {stats['total_tests']}

"""
    
    # Add security comparison
    report_content += """## Security Analysis

| Protocol | Post-Quantum | Malicious Security | Production Ready |
|----------|--------------|-------------------|------------------|
| Microsoft-APSI | ✅ | ✅ | ✅ |
| OPRF-PSI | ❌ | ❌ | Partial |
| Piano-PSI | ✅ | Partial | Research |
| VOLE-PSI | ✅ | ✅ | Research |
| Circuit-PSI | ✅ | ✅ | Research |

## Recommendations

1. **Microsoft APSI** - Best for production deployments requiring proven security
2. **Piano-PSI** - Optimal for large-scale applications with sublinear scaling needs
3. **OPRF-PSI** - Good baseline for educational and research purposes

---
*Report generated by PSI Performance Dashboard v1.0*
"""
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report_content)
        print(f"✅ Detailed report exported to: {filename}")
    except Exception as e:
        print(f"❌ Export failed: {e}")

def quick_performance_test():
    """Run a quick performance test with Microsoft APSI"""
    print("🚀 Running Quick Microsoft APSI Performance Test...")
    
    try:
        apsi = MicrosoftAPSI(security_parameter=128, protocol_mode="rsa-psi")
        
        # Small test sets
        client_set = set(range(10, 60))  # 50 elements
        server_set = set(range(30, 100))  # 70 elements
        
        print(f"Client set size: {len(client_set)}")
        print(f"Server set size: {len(server_set)}")
        print(f"Expected intersection: {len(client_set.intersection(server_set))}")
        
        result = apsi.compute_intersection(client_set, server_set)
        
        print(f"\n📊 Results:")
        print(f"  Execution time: {result.execution_time_ms:.2f} ms")
        print(f"  Intersection size: {result.intersection_size}")
        print(f"  Communication cost: {result.communication_cost_bytes:,} bytes")
        print(f"  Correctness: {'✅ CORRECT' if result.is_correct else '❌ INCORRECT'}")
        
        return result
        
    except Exception as e:
        print(f"❌ Quick test failed: {e}")
        return None

print("✅ Advanced analysis functions loaded")
print("📋 Available functions:")
print("  - analyze_scalability_trends(results)")
print("  - export_detailed_report(results, filename)")
print("  - quick_performance_test()")

✅ Advanced analysis functions loaded
📋 Available functions:
  - analyze_scalability_trends(results)
  - export_detailed_report(results, filename)
  - quick_performance_test()


## 🎯 Quick Test Section

Run a quick Microsoft APSI test to verify the system is working:

In [18]:
# Run a quick performance test
quick_result = quick_performance_test()

if quick_result:
    print("\n🎉 System verification successful!")
    print("You can now use the interactive dashboard above for comprehensive benchmarks.")
else:
    print("\n⚠️ Please check the APSI C++ executable and try again.")

2025-09-18 19:44:47,761 - INFO - 🔐 Microsoft-APSI-RSA-PSI initialized with 128-bit security
2025-09-18 19:44:47,762 - INFO -    Protocol mode: rsa-psi
2025-09-18 19:44:47,763 - INFO -    C++ executable: d:\pyproject\kccrypt\secureComputation\Release\apsi_only_test.exe
2025-09-18 19:44:47,763 - INFO - 🚀 Starting Microsoft-APSI-RSA-PSI protocol
2025-09-18 19:44:47,764 - INFO -    Client set size: 50
2025-09-18 19:44:47,765 - INFO -    Server set size: 70
2025-09-18 19:44:47,766 - INFO -    Using C++ executable: True
2025-09-18 19:44:47,766 - INFO -    Step 1: Preparing input files...
2025-09-18 19:44:47,769 - INFO -    Prepared input files: C:\Users\C00019~1\AppData\Local\Temp\tmp8g2uukf1_client.txt, C:\Users\C00019~1\AppData\Local\Temp\tmpoc1pfuz1_server.txt
2025-09-18 19:44:47,771 - INFO -    Step 2: Calling Microsoft APSI C++ implementation...
2025-09-18 19:44:47,772 - INFO -    Executing C++ APSI: d:\pyproject\kccrypt\secureComputation\Release\apsi_only_test.exe --client-file C:\User

🚀 Running Quick Microsoft APSI Performance Test...
Client set size: 50
Server set size: 70
Expected intersection: 30


2025-09-18 19:44:48,069 - INFO -    ✅ C++ APSI completed successfully
2025-09-18 19:44:48,078 - INFO -    Found 30 intersection elements
2025-09-18 19:44:48,078 - INFO -    ✅ Microsoft-APSI-RSA-PSI completed in 314.94 ms
2025-09-18 19:44:48,079 - INFO -    Intersection size: 30
2025-09-18 19:44:48,079 - INFO -    Communication cost (estimated): 3,280 bytes
2025-09-18 19:44:48,080 - INFO -    Correctness: ✅ CORRECT



📊 Results:
  Execution time: 314.94 ms
  Intersection size: 30
  Communication cost: 3,280 bytes
  Correctness: ✅ CORRECT

🎉 System verification successful!
You can now use the interactive dashboard above for comprehensive benchmarks.


## 📚 Educational Resources

### 🔍 Understanding PSI Protocols

**Private Set Intersection (PSI)** allows two parties to compute the intersection of their sets without revealing any information about elements not in the intersection.

#### Protocol Overviews:

1. **Microsoft APSI (Asymmetric PSI)**
   - Based on Ring Learning With Errors (RLWE)
   - Supports unbalanced sets efficiently
   - Production-ready C++ implementation
   - Post-quantum secure

2. **OPRF-PSI (Oblivious Pseudo-Random Function PSI)**
   - Uses blinding and pseudo-random functions
   - Good for balanced sets
   - Based on Diffie-Hellman assumptions

3. **Piano-PSI**
   - Sublinear communication complexity O(√n·m)
   - Optimal for very large datasets
   - Uses advanced PIR techniques

4. **VOLE-PSI (Vector Oblivious Linear Evaluation PSI)**
   - High performance for large-scale applications
   - Post-quantum secure (LWE-based)
   - Supports batch processing

5. **Circuit-PSI**
   - Most flexible, supports complex variants
   - Higher communication overhead
   - Malicious security model

### 📖 Further Reading

- [Microsoft APSI Library](https://github.com/microsoft/APSI)
- [PSI Survey Paper](https://eprint.iacr.org/2019/723.pdf)
- [VOLE-PSI Implementation](https://github.com/Visa-Research/volepsi)

---

## 🔧 Technical Notes

This dashboard integrates with the `PrivateSetInter.py` module to provide:
- Real Microsoft APSI C++ library execution
- Comprehensive performance benchmarking
- Interactive parameter configuration
- Advanced visualization capabilities

**Author:** kn1ghtc | **Platform:** Honor Security Research Platform  
**Version:** 1.0 | **Date:** 2024-09-18